# Add error-correction based on model eval results

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pylab as plt

In [5]:
import sys
sys.path.append('/home/catskills/Desktop/openasr20/end2end_asr_pytorch')
import os
os.environ['IN_JUPYTER']='True'

In [6]:
from glob import glob
import json
import numpy as np

In [7]:
language='amharic'
stage='NIST'

In [9]:
with open(f'analysis/{language}/eval.json', 'r') as f:
    R=json.load(f)

In [10]:
R[0:2]

[[0, 0, 'እ', 'እ', 'SUCCESS'], [0, 0, 'እ', 'እ', 'SUCCESS']]

In [11]:
len(R)

7492

In [12]:
E=[x for x in R if x[-1] != 'SUCCESS']

In [13]:
len(E)

2810

In [14]:
len(E)/len(R)

0.37506673785371064

In [113]:
from tokenizer import tokenizer

In [63]:
R1=[(tokenizer(gold), tokenizer(pred), success=='SUCCESS') for x,y,gold,pred,success in R]

In [107]:
len(R1), len(R)

(7492, 7492)

In [64]:
R1[0]

(['እ'], ['እ'], True)

In [65]:
vocab=[]
for gold,pred,x in R1:
    vocab.extend(gold)
    vocab.extend(pred)
vocab=list(set(vocab))
len(vocab)

12055

In [66]:
vocab[0:10]

['ጋዋ',
 'አይመችም',
 'እየተማረች',
 'እእለኩ',
 'በተስፋ',
 'ይርጋለም',
 'ምንቺ',
 'ተነምጠን',
 'እኮም',
 'በስደም']

In [67]:
M={}
for gold,pred,x in R1:
    for y in gold:
        M[y]=y

In [68]:
len(M)

6440

In [69]:
for gold,pred,x in R1:
    if len(gold)==len(pred):
        for wgold,wpred in zip(gold,pred):
            if wpred not in M:
                M[wpred]=wgold

In [70]:
len(M)

10491

In [71]:
Z=[x for x in vocab if x not in M]

In [72]:
len(Z)

1564

In [73]:
Z[0:10]

['ተነምጠን', 'በስደም', 'ሄድ', 'እሸት', 'አላዳት', 'ተለያያያሁ', 'ከዎ', 'እምጣ', 'እንደትከውጭ', 'ነጅ']

In [74]:
for x in Z:
    M[x]=''

In [75]:
len(M)

12055

In [114]:
from error_correct import error_correct

In [87]:
# Do no harm
for gold,pred,success in R1:
    if not success:
        continue
    repred=error_correct(pred, M)
    if repred != pred:
        print('whoops', repred)
        break

In [109]:
# First pass
fixed=[]
harder=[]
for gold,pred,success in R1:
    repred=error_correct(pred, M)
    if repred == gold:
        fixed.append((gold,pred))
    else:
        harder.append((gold,pred,repred))

In [110]:
len(fixed), len(harder), len(R1)

(5952, 1540, 7492)

In [105]:
len(R)

7492

In [112]:
len(E)-len(harder)

1270

In [115]:
with open(f'analysis/{language}/afterburner.json', 'w') as f:
    json.dump(M, f)